In [1]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \n    \'// setup cpp code highlighting\\n\'\n    \'IPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\'\n    \'IPython.CodeCell.options_default.highlight_modes["text/x-cmake"] = {\\\'reg\\\':[/^%%cmake/]} ;\'\n)\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE, STDOUT, check_output\nimport html\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\nimport time\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    parser.add_argument("--under-spoiler-threshold", type=int, default=None)\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                comment_prefix = "%" + line_comment_start\n                if line.startswith(comment_prefix):\n                    cmds.append(\'#\' + line[len(comment_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        if cmd.startswith(\'#\'):\n            display(Markdown("\\#\\#\\#\\# `%s`" % cmd[1:]))\n        else:\n            display(Markdown("Run: `%s`" % cmd))\n            if args.under_spoiler_threshold:\n                out = check_output(cmd, stderr=STDOUT, shell=True, universal_newlines=True)\n                out = out[:-1] if out.endswith(\'\\n\') else out\n                out = html.escape(out)\n                if len(out.split(\'\\n\')) > args.under_spoiler_threshold:\n                    out = "<details> <summary> output </summary> <pre><code>%s</code></pre></details>" % out\n                elif out:\n                    out = "<pre><code>%s</code></pre>" % out\n                if out:\n                    display(HTML(out))\n            else:\n                get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef cmake(fname, cell):\n    save_file(fname, cell, "#")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    fname = fname or "makefile"\n    assert fname.endswith("makefile")\n    save_file(fname, cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    line = line.strip() \n    if line[0] == \'#\':\n        display(Markdown(line[1:].strip()))\n    else:\n        try:\n            expr, comment = line.split(" #")\n            display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n        except:\n            display(Markdown("{} = {}".format(line, eval(line))))\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def wait_stop(self, timeout):\n        for i in range(int(timeout * 10)):\n            wpid, status = os.waitpid(self.pid, os.WNOHANG)\n            if wpid != 0:\n                return True\n            time.sleep(0.1)\n        return False\n        \n    def close(self, timeout=3):\n        self.inq_f.close()\n        if not self.wait_stop(timeout):\n            os.kill(self.get_pid(), signal.SIGKILL)\n            os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# Аттрибуты файлов и файловых дескрипторов

<p><a href="https://www.youtube.com/watch?v=bMmE7PPA1LQ&list=PLjzMm8llUm4AmU6i_hPU0NobgA4VsBowc&index=12" target="_blank">
    <h3>Видеозапись семинара</h3>
</a></p>


[Ридинг Яковлева про stat](https://github.com/victor-yacovlev/mipt-diht-caos/tree/master/practice/stat_fcntl) 
<br>[Ридинг Яковлева про работу с директориями, временем и еще несколькими вещами](https://github.com/victor-yacovlev/mipt-diht-caos/tree/master/practice/posix_dirent_time)
<br>[Ридинг Яковлева про FUSE](https://github.com/victor-yacovlev/mipt-diht-caos/tree/master/practice/fuse)


Сегодня в программе:
* <a href="#fs_posix" style="color:#856024"> Работа с файловой системой POSIX </a>
  * <a href="#opendir" style="color:#856024"> Просмотр содержимого директории c фильтрацией по регулярке </a>
  * <a href="#glob" style="color:#856024"> glob или история о том, как вы пишете *.cpp в терминале </a>
  * <a href="#fs_stat" style="color:#856024"> Информация о файловой системе. </a>
* <a href="#stat" style="color:#856024"> Атрибуты файлов и разные способы их получения </a>  
  * <a href="#time" style="color:#856024"> Извлечем время доступа из атрибутов файла </a>  
* <a href="#link" style="color:#856024"> Ссылки жесткие и символические </a>  
* <a href="#fds" style="color:#856024"> Атрибуты файловых дескрипторов </a> 
* <a href="#fusepy" style="color:#856024"> FUSE: Python + fusepy.  Примонтируем json как read-only файловую систему. </a>


## <a name="fs_posix"></a> Работа с файловой системой в POSIX



## <a name="opendir"></a> Просмотр содержимого директории с фильтрацией по регулярке


In [6]:
%%cpp traverse_dir.c
%run gcc -Wall -Werror -fsanitize=address traverse_dir.c -lpthread -o traverse_dir.exe
%run ./traverse_dir.exe ..

#include <stdio.h>
#include <dirent.h>
#include <assert.h>
#include <fnmatch.h>

int main(int argc, char** argv) {
    assert(argc == 2);
    const char* dir_path = argv[1];
    DIR *pDir = opendir(dir_path);
    if (pDir == NULL) {
        fprintf(stderr, "Cannot open directory '%s'\n", dir_path);
        return 1;
    }
    int limit = 4;
    for (struct dirent *pDirent; (pDirent = readdir(pDir)) != NULL && limit > 0;) {
        // + Регулярочки
        if (fnmatch("sem0[12]*", pDirent->d_name, 0) == 0) {
            printf("%s\n", pDirent->d_name);
            --limit;
        }
    }
    closedir(pDir);
    return 0;
}

Run: `gcc -Wall -Werror -fsanitize=address traverse_dir.c -lpthread -o traverse_dir.exe`

Run: `./traverse_dir.exe ..`

sem02-int-float-string-encoding
sem01-instruments-compilation-libc


## <a name="glob"></a> glob или история о том, как вы пишете *.cpp в терминале

Это не совсем про файловую систему, но тем не менее интересно

glob хорошо сочетается с exec, пример тут http://man7.org/linux/man-pages/man3/glob.3.html

In [7]:
%%cpp traverse_dir.c
%run gcc -Wall -Werror -fsanitize=address traverse_dir.c -lpthread -o traverse_dir.exe
%run ./traverse_dir.exe .. | head -n 5

#include <stdio.h>
#include <assert.h>
#include <glob.h>

int main() {
    glob_t globbuf = {0};
    glob("*.c", 0, NULL, &globbuf);
    glob("../*/*.c", GLOB_APPEND, NULL, &globbuf);
    for (char** path = globbuf.gl_pathv; *path; ++path) {
        printf("%s\n", *path);;
    }
    globfree(&globbuf);
    return 0;
}

Run: `gcc -Wall -Werror -fsanitize=address traverse_dir.c -lpthread -o traverse_dir.exe`

Run: `./traverse_dir.exe .. | head -n 5`

traverse_dir.c
../extra-c-basics_2020-2021/eucl.c
../extra-c-basics_2020-2021/main.c
../extra-c-basics_2020-2021/merge.c
../extra-c-basics_2020-2021/task.c


In [8]:
import glob
glob.glob("../*/*.c")[:4]

['../sem02-int-float-string-encoding/lib2.c',
 '../sem02-int-float-string-encoding/main.c',
 '../sem02-int-float-string-encoding/bitcast.c',
 '../sem02-int-float-string-encoding/size.c']

## <a name="fs_stat"></a> Информация о файловой системе

In [19]:
%%cpp fs_stat.c
%run gcc -Wall -Werror -fsanitize=address fs_stat.c -lpthread -o fs_stat.exe
%run ./fs_stat.exe /
%run ./fs_stat.exe /dev/shm
%run ./fs_stat.exe /dev

#include <stdio.h>
#include <sys/statvfs.h>
#include <assert.h>

    
int main(int argc, char** argv) {
    assert(argc == 2);
    const char* dir_path = argv[1];
    struct statvfs stat;
    statvfs(dir_path, &stat);
    
    printf("Free 1K-blocks %lu/%lu", stat.f_bavail * stat.f_bsize / 1024, stat.f_blocks * stat.f_bsize / 1024);
    return 0;
}

Run: `gcc -Wall -Werror -fsanitize=address fs_stat.c -lpthread -o fs_stat.exe`

Run: `./fs_stat.exe /`

Free 1K-blocks 80294736/102168536

Run: `./fs_stat.exe /dev/shm`

Free 1K-blocks 1017424/1017424

Run: `./fs_stat.exe /dev`

Free 1K-blocks 989252/989252

In [18]:
!df | grep -E "Filesys|/dev$|/dev/shm$|/$"

Filesystem     1K-blocks     Used Available Use% Mounted on
udev              989252        0    989252   0% /dev
/dev/sda5      102168536 16640904  80294736  18% /
tmpfs            1017424        0   1017424   0% /dev/shm


## <a name="stat"></a> Атрибуты файлов и разные способы их получения

Сигнатуры функций, с помощью которых можно получить аттрибуты

```c
#include <sys/types.h>
#include <sys/stat.h>
#include <unistd.h>

int stat(const char *pathname, struct stat *buf);
int fstat(int fd, struct stat *buf);
int lstat(const char *pathname, struct stat *buf);

#include <fcntl.h>           /* Definition of AT_* constants */
#include <sys/stat.h>

int fstatat(int dirfd, const char *pathname, struct stat *buf,
		   int flags);
```


Описание структуры из `man 2 stat`:

```c
struct stat {
   dev_t     st_dev;         /* ID of device containing file */
   ino_t     st_ino;         /* inode number */
   mode_t    st_mode;        /* protection */
   nlink_t   st_nlink;       /* number of hard links */
   uid_t     st_uid;         /* user ID of owner */
   gid_t     st_gid;         /* group ID of owner */
   dev_t     st_rdev;        /* device ID (if special file) */
   off_t     st_size;        /* total size, in bytes */
   blksize_t st_blksize;     /* blocksize for filesystem I/O */
   blkcnt_t  st_blocks;      /* number of 512B blocks allocated */

   /* Since Linux 2.6, the kernel supports nanosecond
      precision for the following timestamp fields.
      For the details before Linux 2.6, see NOTES. */

   struct timespec st_atim;  /* time of last access */
   struct timespec st_mtim;  /* time of last modification */
   struct timespec st_ctim;  /* time of last status change */

#define st_atime st_atim.tv_sec      /* Backward compatibility */
#define st_mtime st_mtim.tv_sec
#define st_ctime st_ctim.tv_sec
};
```

Особый интерес будет представлять поле `.st_mode`

Биты соответствующие маскам:
* `0170000` - тип файла.

  Эти биты стоит рассматривать как одно число, по значению которого можно определить тип файла. Сравнивая это число с:  
    * `S_IFSOCK   0140000   socket`
    * `S_IFLNK    0120000   symbolic link`
    * `S_IFREG    0100000   regular file`
    * `S_IFBLK    0060000   block device`
    * `S_IFDIR    0040000   directory`
    * `S_IFCHR    0020000   character device`
    * `S_IFIFO    0010000   FIFO`
* `0777` - права на файл.

  Эти биты можно рассматривать как независимые биты, каджый из которых отвечает за право (пользователя, его группы, всех остальных) (читать/писать/выполнять) файл.

**fstat** - смотрит по файловому дескриптору

In [24]:
%%cpp stat.h

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>

void describe_stat_st_mode(const struct stat* s) {   
    printf("is regular: %s    ", ((s->st_mode & S_IFMT) == S_IFREG) ? "yes" : "no "); // can use predefined mask
    printf("is directory: %s    ", S_ISDIR(s->st_mode) ? "yes" : "no "); // or predefined macro
    printf("is symbolic link: %s\n", S_ISLNK(s->st_mode) ? "yes" : "no "); 
}

In [32]:
%%cpp stat.c
%run gcc stat.c -o stat.exe
%run rm -rf tmp && mkdir tmp && touch tmp/a && ln -s ./a tmp/a_link && mkdir tmp/dir
%run for i in tmp/a tmp/dir tmp/a_link ; do (printf "%10s    " $i ; ./stat.exe < $i); done

#include "stat.h"
#include <assert.h>

int main(int argc, char *argv[])
{   
    struct stat s;
    fstat(0, &s); // get stat for stdin
    describe_stat_st_mode(&s); 
    return 0;
}

Run: `gcc stat.c -o stat.exe`

Run: `rm -rf tmp && mkdir tmp && touch tmp/a && ln -s ./a tmp/a_link && mkdir tmp/dir`

Run: `for i in tmp/a tmp/dir tmp/a_link ; do (printf "%10s    " $i ; ./stat.exe < $i); done`

     tmp/a    is regular: yes    is directory: no     is symbolic link: no 
   tmp/dir    is regular: no     is directory: yes    is symbolic link: no 
tmp/a_link    is regular: yes    is directory: no     is symbolic link: no 


Обратите внимание, что для симлинки результат показан как для регулярного файла, на который она ссылается.
Тут, вероятно, замешан bash, который открывает файл на месте stdin для нашей программы. Видно, что он проходит по симлинкам.

**stat** - смотри по имени файла, следует по симлинкам

In [33]:
%%cpp stat.c
%run gcc stat.c -o stat.exe
%run rm -rf tmp && mkdir tmp && touch tmp/a && ln -s ./a tmp/a_link && mkdir tmp/dir
%run for i in tmp/a tmp/dir tmp/a_link ; do (printf "%10s    " $i ; ./stat.exe $i); done

#include "stat.h"
#include <assert.h>

int main(int argc, char *argv[])
{   
    assert(argc == 2);
    struct stat s;
    stat(argv[1], &s); // Следует по симлинкам
    describe_stat_st_mode(&s); 
    return 0;
}

Run: `gcc stat.c -o stat.exe`

Run: `rm -rf tmp && mkdir tmp && touch tmp/a && ln -s ./a tmp/a_link && mkdir tmp/dir`

Run: `for i in tmp/a tmp/dir tmp/a_link ; do (printf "%10s    " $i ; ./stat.exe $i); done`

     tmp/a    is regular: yes    is directory: no     is symbolic link: no 
   tmp/dir    is regular: no     is directory: yes    is symbolic link: no 
tmp/a_link    is regular: yes    is directory: no     is symbolic link: no 


Обратите внимание, что для симлинки результат показан как для регулярного файла, на который она ссылается.

**lstat** - смотрит по имени файла, не следует по симлинкам.

In [34]:
%%cpp stat.c
%run gcc stat.c -o stat.exe
%run rm -rf tmp && mkdir tmp && touch tmp/a && ln -s ./a tmp/a_link && mkdir tmp/dir
%run for i in tmp/a tmp/dir tmp/a_link ; do (printf "%10s    " $i ; ./stat.exe $i); done

#include "stat.h"
#include <assert.h>

int main(int argc, char *argv[])
{   
    assert(argc == 2);
    struct stat s;
    lstat(argv[1], &s); // Не следует по симлинкам, то есть можно узнать stat самого файла симлинки
    describe_stat_st_mode(&s); 
    return 0;
}

Run: `gcc stat.c -o stat.exe`

Run: `rm -rf tmp && mkdir tmp && touch tmp/a && ln -s ./a tmp/a_link && mkdir tmp/dir`

Run: `for i in tmp/a tmp/dir tmp/a_link ; do (printf "%10s    " $i ; ./stat.exe $i); done`

     tmp/a    is regular: yes    is directory: no     is symbolic link: no 
   tmp/dir    is regular: no     is directory: yes    is symbolic link: no 
tmp/a_link    is regular: no     is directory: no     is symbolic link: yes


Сейчас результат для симлинки показан как для самой симлинки. Так как используем специальную функцию.

**open(...O_NOFOLLOW | O_PATH) + fstat** - открываем файл так, чтобы не следовать по симлинкам и далее смотрим его stat

Кстати, открываем не очень честно. С опцией O_PATH нельзя потом применять read, write и еще некоторые операции.

In [98]:
%%cpp stat.c
%run gcc stat.c -o stat.exe
%run rm -rf tmp && mkdir tmp && touch tmp/a && ln -s ./a tmp/a_link && mkdir tmp/dir
%run for i in tmp/a tmp/dir tmp/a_link ; do (printf "%10s    " $i ; ./stat.exe $i); done

#define _GNU_SOURCE // need for O_PATH

#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <assert.h>

int main(int argc, char *argv[])
{   
    assert(argc == 2);
    struct stat s;
    int fd = open(argv[1], O_RDONLY | O_NOFOLLOW | O_PATH);
    assert(fd >= 0);
    fstat(fd, &s); 
    printf("is regular: %s    ", ((s.st_mode & S_IFMT) == S_IFREG) ? "yes" : "no "); // can use predefined mask
    printf("is directory: %s    ", S_ISDIR(s.st_mode) ? "yes" : "no "); // or predefined macro
    printf("is symbolic link: %s\n", S_ISLNK(s.st_mode) ? "yes" : "no ");
    close(fd);
    return 0;
}

Run: `gcc stat.c -o stat.exe`

Run: `rm -rf tmp && mkdir tmp && touch tmp/a && ln -s ./a tmp/a_link && mkdir tmp/dir`

Run: `for i in tmp/a tmp/dir tmp/a_link ; do (printf "%10s    " $i ; ./stat.exe $i); done`

     tmp/a    is regular: yes    is directory: no     is symbolic link: no 
   tmp/dir    is regular: no     is directory: yes    is symbolic link: no 
tmp/a_link    is regular: no     is directory: no     is symbolic link: yes


Здесь тоже результат показан для самой симлинки, поскольку вызываем open со специальными опциями, чтобы не следовать по симлинкам.

**Поэтому важно понимать, какое поведение вы хотите и использовать stat, fstat или lstat**

## <a name="time"></a> Извлечем время доступа из атрибутов файла

Работа с временем это на самом деле отдельная большая тема. Три года назад на нее был [целый семинар](https://github.com/yuri-pechatnov/caos/tree/master/caos_2019-2020/sem28-unix-time).

In [96]:
%%cpp stat.c
%run gcc stat.c -o stat.exe
%run rm -rf tmp && mkdir tmp &&  touch tmp/a && sleep 1.1 &&  ln -s ./a tmp/a_link && sleep 1.1  && mkdir tmp/dir
%run for i in tmp/a tmp/dir tmp/a_link ; do (printf "%10s    " $i ; ./stat.exe < $i); done

#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <stdio.h>
#include <assert.h>
#include <time.h>


char* pretty_time(const struct timespec* timespec_time) {
    // Whole function is not thread safe because of buf.
    struct tm tm_time;
    assert(localtime_r(&timespec_time->tv_sec, &tm_time)); // Thread-safe
    static char buf[100];
    assert(strftime(buf, sizeof(buf), "%H:%M:%S", &tm_time) > 0); // Thread-safe
    return buf; 
}

int main(int argc, char *argv[])
{   
    struct stat s;
    fstat(0, &s); 
    printf("modification time: %s   ", pretty_time(&s.st_mtim));
    printf("access time: %s\n", pretty_time(&s.st_atim));
    return 0;
}

Run: `gcc stat.c -o stat.exe`

Run: `rm -rf tmp && mkdir tmp &&  touch tmp/a && sleep 1.1 &&  ln -s ./a tmp/a_link && sleep 1.1  && mkdir tmp/dir`

Run: `for i in tmp/a tmp/dir tmp/a_link ; do (printf "%10s    " $i ; ./stat.exe < $i); done`

     tmp/a    modification time: 23:38:03   access time: 23:38:03
   tmp/dir    modification time: 23:38:05   access time: 23:38:05
tmp/a_link    modification time: 23:38:03   access time: 23:38:03


## <a name="link"></a> Ссылки жесткие и символические

In [59]:
!(rm lexmpl_*) 2> /dev/null
!touch lexmpl_ordinary.txt
!touch lexmpl_x.txt
!link lexmpl_x.txt lexmpl_x_hard.txt
!ln -s lexmpl_x.txt ./lexmpl_x_sym.txt
!ls -la lexmpl_x*

-rw-rw-r-- 2 pechatnov pechatnov  0 сен 29 23:19 lexmpl_x_hard.txt
lrwxrwxrwx 1 pechatnov pechatnov 12 сен 29 23:19 lexmpl_x_sym.txt -> lexmpl_x.txt
-rw-rw-r-- 2 pechatnov pechatnov  0 сен 29 23:19 lexmpl_x.txt


In [69]:
!echo "Hello" > lexmpl_x.txt
!ls -la lexmpl_*

-rw-rw-r-- 1 pechatnov pechatnov  0 сен 29 23:19 lexmpl_ordinary.txt
-rw-rw-r-- 2 pechatnov pechatnov  6 сен 29 23:21 lexmpl_x_hard.txt
lrwxrwxrwx 1 pechatnov pechatnov 12 сен 29 23:19 lexmpl_x_sym.txt -> lexmpl_x.txt
-rw-rw-r-- 2 pechatnov pechatnov  6 сен 29 23:21 lexmpl_x.txt


## <a name="fds"></a> Атрибуты файловых дескрипторов

`fcntl(fd, F_GETFL, 0)`, флаги `O_RDWR`, `O_RDONLY`, `O_WRONLY`, ...

In [99]:
%%cpp fcntl_open_flags.cpp
%run gcc fcntl_open_flags.cpp -o fcntl_open_flags.exe
%run ./fcntl_open_flags.exe

#ifndef _GNU_SOURCE
  #define _GNU_SOURCE
#endif
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <fcntl.h>

void describe_fd_impl(const char* prefix, int fd) {
    if (fd < 0) { // Check that fd is valid.
        perror("open"); 
        abort();
    }
    int ret = fcntl(fd, F_GETFL, 0); // Get flags here!
    int mask = O_RDONLY | O_WRONLY | O_RDWR;
#define flag_cond_str_expanded(flag, name) ((ret & mask) == flag ? name : "")
#define flag_cond_str(flag) flag_cond_str_expanded(flag, #flag)
    printf("%55s:   %s%s%s\n", prefix, flag_cond_str(O_RDONLY), flag_cond_str(O_WRONLY), flag_cond_str(O_RDWR));
}

int main() {
#define describe(op) describe_fd_impl(#op, op)
    describe(open("fcntl_open_flags.1", O_CREAT | O_WRONLY, 0664));
    describe(open("fcntl_open_flags.2", O_CREAT | O_RDWR, 0664));
    describe(open("fcntl_open_flags.2", O_RDONLY));
    return 0;
}

Run: `gcc fcntl_open_flags.cpp -o fcntl_open_flags.exe`

Run: `./fcntl_open_flags.exe`

   open("fcntl_open_flags.1", O_CREAT | O_WRONLY, 0664):   O_WRONLY
     open("fcntl_open_flags.2", O_CREAT | O_RDWR, 0664):   O_RDWR
                   open("fcntl_open_flags.2", O_RDONLY):   O_RDONLY


In [100]:
%%cpp istty.c
%run gcc istty.c -o istty.exe
%run ./istty.exe > a.txt
%run ./istty.exe 

#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>

int main(int argc, char *argv[])
{
    if (isatty(STDOUT_FILENO)) {
        fprintf(stderr, "\033[0;31mIt's terminal\033[0m\n");
    } else {
        fprintf(stderr, "It's NOT terminal\n");
    }
    return 0;
}

Run: `gcc istty.c -o istty.exe`

Run: `./istty.exe > a.txt`

It's NOT terminal


Run: `./istty.exe`

It's terminal


## <a name="fusepy"></a> FUSE: Python + fusepy.  Примонтируем json как read-only файловую систему. </a>



[FUSE на wiki](https://ru.wikipedia.org/wiki/FUSE_(модуль_ядра))

![FUSE](https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/FUSE_structure.svg/490px-FUSE_structure.svg.png)


https://habr.com/ru/post/315654/ - на питоне

https://engineering.facile.it/blog/eng/write-filesystem-fuse/
  
  

Установка: `pip2 install --user fusepy`

In [101]:
%%writefile fuse_json.py
from __future__ import print_function

import logging
import os
import json
from errno import EIO, ENOENT, EROFS
from stat import S_IFDIR, S_IFREG
from sys import argv, exit
from time import time

from fuse import FUSE, FuseOSError, LoggingMixIn, Operations

NOW = time()

DIR_ATTRS = dict(st_mode=(S_IFDIR | 0o555), st_nlink=2)
FILE_ATTRS = dict(st_mode=(S_IFREG | 0o444), st_nlink=1)

def find_json_path(j, path):
    for part in path.split('/'):
        if len(part) > 0:
            if part == '__json__':
                return json.dumps(j)
            if part not in j:
                return None
            j = j[part]
    return j
    

class FuseOperations(LoggingMixIn, Operations):

    def __init__(self, j):
        self.j = j
        self.fd = 0

    def open(self, path, flags):
        self.fd += 1
        return self.fd

    def read(self, path, size, offset, fh):
        logging.debug("Read %r %r %r", path, size, offset)
        node = find_json_path(self.j, path)
        if not isinstance(node, str):
            raise FuseOSError(EIO)
        return node[offset:offset + size]

    def readdir(self, path, fh):
        logging.debug("Readdir %r %r", path, fh)
        node = find_json_path(self.j, path)
        if node is None:
            raise FuseOSError(EROFS)
        return ['.', '..', '__json__'] + list(node.keys())

    def getattr(self, path, fh=None):
        node = find_json_path(self.j, path)
        if isinstance(node, dict):
            return DIR_ATTRS
        elif isinstance(node, str):
            attrs = dict(FILE_ATTRS)
            attrs["st_size"] = len(node)
            return attrs
        else:
            raise FuseOSError(ENOENT)

if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO)
    j = {
        'a': 'b',
        'c': {
            'c1': '234'
        }
    }
    FUSE(FuseOperations(j), "./fuse_json", foreground=True)

Writing fuse_json.py


In [102]:
!mkdir fuse_json 2>&1 | grep -v "File exists" || true
a = TInteractiveLauncher("python2 fuse_json.py example.txt fuse_json 2>&1")

In [120]:
%%bash
exec 2>&1 ; set -v ; set -o pipefail

ls -la fuse_json                      # List dir. 

tree fuse_json --noreport             # Recursively list dirs (install: sudo apt install tree). 

cat fuse_json/__json__    && echo     #

cat fuse_json/a           && echo     #

cat fuse_json/c/__json__  && echo     #


ls -la fuse_json                      # List dir. 
total 8
drwxrwxr-x 2 pechatnov pechatnov 4096 сен 29 23:41 .
drwxrwxr-x 6 pechatnov pechatnov 4096 сен 29 23:48 ..

tree fuse_json --noreport             # Recursively list dirs (install: sudo apt install tree). 
fuse_json

cat fuse_json/__json__    && echo     #
cat: fuse_json/__json__: No such file or directory

cat fuse_json/a           && echo     #
cat: fuse_json/a: No such file or directory

cat fuse_json/c/__json__  && echo     #
cat: fuse_json/c/__json__: No such file or directory


CalledProcessError: Command 'b'exec 2>&1 ; set -v ; set -o pipefail\n\nls -la fuse_json                      # List dir. \n\ntree fuse_json --noreport             # Recursively list dirs (install: sudo apt install tree). \n\ncat fuse_json/__json__    && echo     #\n\ncat fuse_json/a           && echo     #\n\ncat fuse_json/c/__json__  && echo     #\n'' returned non-zero exit status 1.

In [116]:
!fusermount -u fuse_json
a.close()